In [15]:
import torch
import pandas as pd
from rouge import Rouge
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import plotly.express as px
import matplotlib.pyplot as plt

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)


In [16]:
data=pd.read_csv('train_hindi_new.csv',engine='python')

In [17]:
data.dropna(inplace=True)

In [18]:
data=data.head(5000)

In [19]:
data=data.reset_index()

In [20]:
data.drop('index',axis=1,inplace=True)

In [21]:
data

,headline,summary,article
0,रिम करेगा दो हजार कर्मचारियों की छंटनी,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...
1,अमेरिका : राष्ट्रपति पद की उम्मीदवार हिलेरी क्...,श्रद्धांजलि सभा के दौरान अचानक खराब हुई हिलेरी...,अमेरिका में राष्ट्रपति पद के लिए डेमोक्रेटिक प...
2,देश के सर्वश्रेष्ठ स्पिनरों में से एक है हरभजन...,पूर्व भारतीय कप्तान अनिल कुंबले का मानना है कि...,पूर्व भारतीय क्रिकेट कप्तान अनिल कुंबले का मान...
3,इंडिगो के स्टाफ ने दिल्ली एयरपोर्ट पर पैसेंजर ...,इंडिगो के ग्राउंड स्टाफ ने यात्री से की हाथापा...,इंडिगो एयरलाइन के प्रेसीडेंट और डायरेक्टर आदित...
4,नव्या नवेली के साथ दोस्ती पर जावेद जाफरी के बे...,मीजान जाफरी ने किया खुलासा\r\nनव्या नंदा को बत...,संजय लीला भंसाली की फिल्म 'मलाल' से बॉलीवुड मे...
...,...,...,...
4995,केंद्रीय कर्मचारियों के महंगाई भत्ते में 6 % क...,सरकार ने महंगाई से परेशान केंद्रीय कर्मचारियों...,सरकार ने महंगाई से परेशान केंद्रीय कर्मचारियों...
4996,उत्तराखंड उप-चुनाव : 40 हजार वोटों से जीते विज...,उत्तराखंड के मुख्यमंत्री विजय बहुगुणा सितारगंज...,उत्तराखंड के मुख्यमंत्री विजय बहुगुणा सितारगंज...
4997,"इस्तीफे के पीछे कोई राजनीति नहीं, 95 साल की मा...",नजीब जंग से आज अरविंद केजरीवाल ने मुलाकात की\r...,दिल्ली के उपराज्यपाल रहे नजीब जंग ने पद से इस्...
4998,किसी भी तरह की लापरवाही के लिए अधिकारियों की ज...,उपराज्यपाल नजीब जंग ने की राष्ट्रीय राजधानी की...,उपराज्यपाल नजीब जंग ने गुरुवार को राष्ट्रीय रा...


In [22]:
x=[]
for i in data['article']:
    x.append(len(i))

In [23]:
data=data.head(30)

In [24]:
text = data['article']
goldsummary = data['summary']

In [25]:
bart_checkpoint = "ai4bharat/IndicBART"

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu" 

In [12]:
tokenizer_bart = AutoTokenizer.from_pretrained(bart_checkpoint, 
                                               do_lower_case=False, 
                                               use_fast=False, 
                                               keep_accents=True)

model_bart = AutoModelForSeq2SeqLM.from_pretrained(bart_checkpoint).to(device)

In [13]:
bos_id = tokenizer_bart._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer_bart._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer_bart._convert_token_to_id_with_added_voc("<pad>")

In [44]:
%%time

SystemSummary = []

for i, input_text in enumerate(text):
    
    inp = tokenizer_bart(input_text, add_special_tokens=False, 
                         truncation=True, return_tensors="pt", 
                         padding='max_length', max_length=1024)['input_ids'].to(device)  
    
    model_output = model_bart.generate(inp, use_cache=True, 
                                        num_beams=4, 
                                        max_length=200, 
                                        min_length=30, 
                                        early_stopping=True, 
                                        pad_token_id=pad_id,
                                        bos_token_id=bos_id, 
                                        eos_token_id=eos_id, 
                                        decoder_start_token_id=tokenizer_bart._convert_token_to_id_with_added_voc("<2en>"))
    
    decoded_output = tokenizer_bart.decode(model_output[0], 
                                    skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=False)
    
    SystemSummary.append(decoded_output)

Wall time: 17min 44s


In [31]:
Summaries = pd.DataFrame(list(zip(text,goldsummary, SystemSummary)), columns =['Articles','GoldSummary', 'BartSummary'])
Summaries

,Articles,GoldSummary,BartSummary
0,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...
1,अमेरिका में राष्ट्रपति पद के लिए डेमोक्रेटिक प...,श्रद्धांजलि सभा के दौरान अचानक खराब हुई हिलेरी...,अमेरिका में राष्ट्रपति पद के लिए डेमोक्रेटिक प...
2,पूर्व भारतीय क्रिकेट कप्तान अनिल कुंबले का मान...,पूर्व भारतीय कप्तान अनिल कुंबले का मानना है कि...,पूर्व भारतीय क्रिकेट कप्तान अनिल कुंबले का मान...
3,इंडिगो एयरलाइन के प्रेसीडेंट और डायरेक्टर आदित...,इंडिगो के ग्राउंड स्टाफ ने यात्री से की हाथापा...,इंडिगो एयरलाइन के प्रेसीडेंट और डायरेक्टर आदित...
4,संजय लीला भंसाली की फिल्म 'मलाल' से बॉलीवुड मे...,मीजान जाफरी ने किया खुलासा\r\nनव्या नंदा को बत...,संजय लीला भंसाली की फिल्म 'मलाल' से बॉलीवुड मे...
5,इस पूरे मामले की जांच उत्तर-प्रदेश सरकार द्वार...,100 समर्थकों के साथ पुलिस अधिकारी के आवास के ब...,इस पूरे मामले की जांच उत्तर-प्रदेश सरकार द्वार...
6,विवादित इस्लामिक प्रचारक ज़ाकिर नाइक के गैर सर...,ज़ाकिर नाइक के 78 बैंक खातों की एनआईए जांच कर ...,विवादित इस्लामिक प्रचारक ज़ाकिर नाइक के गैर सर...
7,राष्ट्रपति प्रतिभा पाटील रविवार सुबह सेशेल्स औ...,राष्ट्रपति प्रतिभा पाटील रविवार सुबह सेशेल्स औ...,राष्ट्रपति प्रतिभा पाटील रविवार सुबह सेशेल्स औ...
8,उन्होंने कहा कि अपने क्षेत्रों की समस्याओं को ...,कहा- जब संसद नहीं चल रही हो तो अपने क्षेत्र मे...,उन्होंने कहा कि अपने क्षेत्रों की समस्याओं को ...
9,Jitne safed baal mere sar aur dhaadi mein hain...,सलमान खान दिखे बुजुर्ग के गेटअप में\r\nउनकी तस...,Jitne safed baal mere sar aur dhaadi mein hain...


In [35]:
rouge = Rouge()
score = rouge.get_scores(Summaries['BartSummary'], Summaries['GoldSummary'], avg=True)
BartRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
BartRouge


,rouge-1,rouge-2,rouge-l
recall,0.613162,0.465373,0.579306
precision,0.386013,0.261637,0.367227
f-measure,0.468503,0.331785,0.444694


In [18]:
print('Article:',Summaries['Articles'][0])
print('======================================================')
print('Gold Summary:',Summaries['GoldSummary'][0])
print('======================================================')
print('Bart Summary:', Summaries['BartSummary'][0])

‘द ग्लोब एंड मेल’ ने कंपनी के नजदीकी सूत्रों के हवाले से प्रकाशित खबर में कहा है कि रिम वैश्विक स्तर पर बड़े पुनर्गठन की तैयारी कर रही है। अगले कुछ सप्ताह में वह वैश्विक स्तर पर 2,000 नौकरियों की कटौती करेगी। रिपोर्ट में कहा गया है कि छंटनी का आंकड़ा इससे भी बड़ा हो सकता है।
‘द ग्लोब एंड मेल’ ने कंपनी के नजदीकी सूत्रों के हवाले से प्रकाशित खबर में कहा है कि रिम वैश्विक स्तर पर बड़े पुनर्गठन की तैयारी कर रही है। अगले कुछ सप्ताह में वह वैश्विक स्तर पर 2,000 नौकरियों की कटौती करेगी। रिपोर्ट में कहा गया है कि छंटनी का आंकड़ा इससे भी बड़ा हो सकता है।
Gold Summary: ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रिसर्च इन मोशन (रिम) अपनी पुनर्गठन योजना के तहत अगले कुछ सप्ताह में वैश्विक स्तर पर 2,000 कर्मचारियों की छंटनी करेगी।
Bart Summary: ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रिसर्च इन मोशन (रिम) अपनी पुनर्गठन योजना के तहत अगले कुछ सप्ताह में वैश्विक स्तर पर 2,000 कर्मचारियों की छंटनी करेगी। ‘द ग्लोब एंड मेल’ ने कंपनी के नजदीकी सूत्रों के हवाले से प्रकाशित खबर में कहा है कि रिम वैश्विक 

In [27]:
t5_checkpoint = "csebuetnlp/mT5_multilingual_XLSum"

tokenizer_t5 = AutoTokenizer.from_pretrained(t5_checkpoint)
model_t5 = AutoModelForSeq2SeqLM.from_pretrained(t5_checkpoint).to(device)

C:\Users\Acer\anaconda3\lib\site-packages\transformers\convert_slow_tokenizer.py:446: UserWarning:

The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.



In [28]:
%%time

SystemSummary1 = []

for i, input_text in enumerate(text):    
    
    input_ids = tokenizer_t5(input_text,
                             return_tensors="pt",
                             padding="max_length",
                             truncation=True,
                             max_length=768)["input_ids"].to(device) 

    model_output = model_t5.generate(input_ids=input_ids,
                                     num_beams=4,
                                     max_length=150,
                                     min_length=30,
                                     no_repeat_ngram_size=2,
                                     early_stopping=True)

    decoded_output = tokenizer_t5.decode(model_output[0],
                                  skip_special_tokens=True,
                                  clean_up_tokenization_spaces=False)

    
    
    SystemSummary1.append(decoded_output)

Wall time: 8min 47s


In [36]:
Summaries['T5Summary'] = SystemSummary1
Summaries.head()

,Articles,GoldSummary,BartSummary,T5Summary
0,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...,कनाडा की कंपनी रिसर्च इन मोशन (रिम) अपनी पुनर्...
1,अमेरिका में राष्ट्रपति पद के लिए डेमोक्रेटिक प...,श्रद्धांजलि सभा के दौरान अचानक खराब हुई हिलेरी...,अमेरिका में राष्ट्रपति पद के लिए डेमोक्रेटिक प...,अमेरिका में राष्ट्रपति पद के लिए डेमोक्रेटिक प...
2,पूर्व भारतीय क्रिकेट कप्तान अनिल कुंबले का मान...,पूर्व भारतीय कप्तान अनिल कुंबले का मानना है कि...,पूर्व भारतीय क्रिकेट कप्तान अनिल कुंबले का मान...,भारतीय क्रिकेट टीम में वापसी करने वाले ऑफ-स्पि...
3,इंडिगो एयरलाइन के प्रेसीडेंट और डायरेक्टर आदित...,इंडिगो के ग्राउंड स्टाफ ने यात्री से की हाथापा...,इंडिगो एयरलाइन के प्रेसीडेंट और डायरेक्टर आदित...,इंडिगो एयरलाइन के डायरेक्टर आदित्य घोष ने रविव...
4,संजय लीला भंसाली की फिल्म 'मलाल' से बॉलीवुड मे...,मीजान जाफरी ने किया खुलासा\r\nनव्या नंदा को बत...,संजय लीला भंसाली की फिल्म 'मलाल' से बॉलीवुड मे...,बॉलीवुड एक्टर मीजान जाफरी और नव्या नवेली नंदा ...


In [37]:
rouge = Rouge()
score = rouge.get_scores(Summaries['T5Summary'], Summaries['GoldSummary'], avg=True)
T5Rouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
T5Rouge

,rouge-1,rouge-2,rouge-l
recall,0.362420,0.199815,0.310326
precision,0.455036,0.272937,0.393526
f-measure,0.397794,0.226780,0.342104


In [38]:
T5Rouge = T5Rouge[['rouge-1','rouge-l', 'rouge-2']]
BartRouge = BartRouge[['rouge-1','rouge-l', 'rouge-2']]
fig = px.bar(T5Rouge*100, x=T5Rouge.index, y=T5Rouge.columns, 
             barmode='group', 
             text_auto='.2s',
             labels={
                     "Algo": "Algorithms",
                     "value": "Rouge Score",
                     "variable": "legend",
                     'index': "Metrics"
                 })

fig.update_layout(width=650,
                  height=400,
                  title={
                  'text': "Score",
                  'y':.96,
                  'x':0.49,
                  'xanchor': 'center',
                  'yanchor': 'top'})

fig.show()

In [39]:
fig = px.bar(BartRouge*100, x=BartRouge.index, y=BartRouge.columns, 
             barmode='group', 
             text_auto='.2s',
             labels={
                     "Algo": "Algorithms",
                     "value": "Rouge Score",
                     "variable": "legend",
                     'index': "Metrics"
                 })
fig.update_layout( width=650,
                  height=400,
                title={
                  'text': "Score",
                  'y':.96,
                  'x':0.49,
                  'xanchor': 'center',
                  'yanchor': 'top'})

fig.show()

In [40]:
print('Article:',Summaries['Articles'][2])
print('======================================================')
print('Gold Summary:',Summaries['GoldSummary'][2])
print('======================================================')
print('Bart Summary:', Summaries['BartSummary'][2])
print('======================================================')
print('T5 Summary:', Summaries['T5Summary'][2])

कुंबले ने देश के लेग स्पिनरों के बारे में कहा, राहुल शर्मा, पीयूष चावला और अमित मिश्रा दौड़ में आगे हैं और टेस्ट प्रारूप के अधिक अनुकूल हैं।
कुंबले ने देश के लेग स्पिनरों के बारे में कहा, राहुल शर्मा, पीयूष चावला और अमित मिश्रा दौड़ में आगे हैं और टेस्ट प्रारूप के अधिक अनुकूल हैं।
कुंबले ने देश के लेग स्पिनरों के बारे में कहा, राहुल शर्मा, पीयूष चावला और अमित मिश्रा दौड़ में आगे हैं और टेस्ट प्रारूप के अधिक अनुकूल हैं।
कुंबले ने देश के लेग स्पिनरों के बारे में कहा, राहुल शर्मा, पीयूष चावला और अमित मिश्रा दौड़ में आगे हैं और टेस्ट प्रारूप के अधिक अनुकूल हैं।
कुंबले ने देश के लेग स्पिनरों के बारे में कहा, राहुल शर्मा, पीयूष चावला और अमित मिश्रा दौड़ में आगे हैं और टेस्ट प्रारूप के अधिक अनुकूल हैं।
कुंबले ने देश के लेग स्पिनरों के बारे में कहा, राहुल शर्मा, पीयूष चावला और अमित मिश्रा दौड़ में आगे हैं और टेस्ट प्रारूप के अधिक अनुकूल हैं।
कुंबले ने देश के लेग स्पिनरों के बारे में कहा, राहुल शर्मा, पीयूष चावला और अमित मिश्रा दौड़ में आगे हैं और टेस्ट प्रारूप के अधिक अनुकूल हैं।
कुंबले ने देश